# TraitGym [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/songlab-cal/TraitGym/blob/main/TraitGym.ipynb)
In this example we will load the Mendelian traits dataset and run variant effect prediction based on euclidean distance of GPN-Animal-Promoter embeddings of the reference and alternate sequences.

## Setup

In [1]:
!pip install -q pyfaidx s3fs git+https://github.com/songlab-cal/gpn.git
!pip install -q -U transformers datasets

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.5/151.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.8/185.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [1]:
# Full clean installation with exact versions
!pip uninstall -y numpy torch transformers tokenizers gpn datasets
!pip install numpy==1.23.5 --no-cache-dir --force-reinstall
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118
!pip install transformers==4.30.1
!pip install tokenizers==0.13.3
!pip install biopython==1.81 pyfaidx==0.8.1 s3fs==2023.6.0
!pip install git+https://github.com/songlab-cal/gpn.git@e0d3870
!pip install datasets==2.14.6
!pip install scikit-learn==1.2.2 seaborn==0.12.2 matplotlib==3.7.1 polars==0.18.3

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: transformers 4.50.3
Uninstalling transformers-4.50.3:
  Successfully uninstalled transformers-4.50.3
Found existing installation: tokenizers 0.21.1
Uninstalling tokenizers-0.21.1:
  Successfully uninstalled tokenizers-0.21.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 198.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.21.0+cu124 requires torch==2.6.0, which is not installed.
timm 1.0.15 requires torch, which is not installed.
peft 0.14.0 requires torch>=1.13.0, which is not installed.
peft 0.14.0 requires transformers, which is not installed.
fastai 2.7.19 requ

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 706.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 98.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lit: filename=lit-15.0.7-py3-none-any.whl size=89990 sha256=88f5fd1f51875b7c8d33d9f01e989d4f7403548c09b745519bbcf855ec68975d
  Stored in directory: /root/.cache/pip/wheels/fc/5d/45/34fe9945d5e45e261134e72284395be36c2d4828af38e2b0fe
Successfully built lit
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: torchvision
    Found existing installation: to

In [1]:
!pip install transformers==4.40.0 \
torch==2.2.0 \
tokenizers==0.19.1 \
accelerate==0.29.3 \
--force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 113.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
!pip install "numpy==1.26.4" "torch==2.2.0" "opencv-python==4.8.0.76"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 821.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch==2.6.0+cu124 torchvision==0.21.0+cu124 torchaudio==2.6.0+cu124 --extra-index-url https://download.pytorch.org/whl/cu124


Found existing installation: torch 2.2.0
Uninstalling torch-2.2.0:
  Successfully uninstalled torch-2.2.0
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import torch

# For PyTorch
print("PyTorch GPU Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")

# For TensorFlow
import tensorflow as tf
print("\nTensorFlow GPU Available:", tf.test.is_gpu_available())
print("TensorFlow GPU Name:", tf.config.list_physical_devices('GPU'))

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


PyTorch GPU Available: True
GPU Name: Tesla T4

TensorFlow GPU Available: True
TensorFlow GPU Name: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
!pip install biopython


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 37.5 MB/s eta 0:00:00


In [5]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.


In [10]:
!pip install "datasets==3.5.0" "fsspec[http]>=2023.1.0,<=2024.12.0" "gcsfs<2025"


  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.12.0-py3-none-any.whl (183 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: gcsfs
    Found existing installation: gcsfs 2025.3.2
    Uninstalling gcsfs-2025.3.2:
      Successfully uninstalled gcsfs-2025.3.2


In [2]:
!pip install git+https://github.com/songlab-cal/gpn.git


  Cloning https://github.com/songlab-cal/gpn.git to /tmp/pip-req-build-wan3y5l0
  Running command git clone --filter=blob:none --quiet https://github.com/songlab-cal/gpn.git /tmp/pip-req-build-wan3y5l0
  Resolved https://github.com/songlab-cal/gpn.git to commit e0d3870a859bbd745de5c49614e0eb3b8bae8cb5
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.5/151.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.8/185.8 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 4.7 MB/s eta 0:00:00
  Created wheel for gpn: filename=gpn-0.6-py3-none-any.whl size=74545 sha256=4bdd872afe8143968669b310ec701efee4671c91fffb9ff47de41037e3b2899d
  Stored in directory: /tmp/pip-ephem-wheel-cache-ot7vlshl/wheels/17/32/63/c

In [4]:
!pip install pyfaidx


In [5]:
from Bio.Seq import Seq
from datasets import load_dataset
import fsspec
import gpn.model  # to register AutoModel
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
from pyfaidx import Fasta
import seaborn as sns
from sklearn.metrics import average_precision_score
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel, TrainingArguments, Trainer
import tempfile

In [6]:
dataset_path = "songlab/TraitGym"
dataset_config = "mendelian_traits"
model_path = "songlab/gpn-animal-promoter"
# in the paper we average with the predictions with the reverse complement
# however for a quick evaluation you can omit this
average_rc = False

## Load dataset

In [7]:
dataset = load_dataset(dataset_path, dataset_config, split="test")
dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.63k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/50.2k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/3380 [00:00<?, ? examples/s]

Dataset({
    features: ['chrom', 'pos', 'ref', 'alt', 'OMIM', 'consequence', 'label', 'tss_dist', 'match_group'],
    num_rows: 3380
})

In [8]:
dataset[0]

{'chrom': '1',
 'pos': 1425822,
 'ref': 'C',
 'alt': 'G',
 'OMIM': None,
 'consequence': 'PLS',
 'label': False,
 'tss_dist': 48,
 'match_group': 'PLS_4'}

In [9]:
# if you just want a dataframe you can load it directly:
# pd.read_parquet("hf://datasets/songlab/TraitGym/mendelian_traits_matched_9/test.parquet")
V = dataset.to_pandas()
V

,chrom,pos,ref,alt,OMIM,consequence,label,tss_dist,match_group
0,1,1425822,C,G,None,PLS,False,48,PLS_4
1,1,1615869,C,T,None,PLS,False,35,PLS_0
2,1,1659060,G,A,None,PLS,False,47,PLS_4
3,1,1659114,A,G,None,PLS,False,101,PLS_5
4,1,2050958,T,C,None,5_prime_UTR_variant,False,149,5_prime_UTR_variant_7
...,...,...,...,...,...,...,...,...,...
3375,X,155613005,C,T,None,PLS,False,52,PLS_52
3376,X,155719093,C,A,None,5_prime_UTR_variant,False,4,5_prime_UTR_variant_101
3377,X,155881342,A,C,None,PLS,False,2,PLS_57
3378,X,155881414,C,T,None,5_prime_UTR_variant,False,35,5_prime_UTR_variant_110


## Load model

In [10]:
# Wrapper to compute a VEP score
class ModelVEP(torch.nn.Module):
    def __init__(self, model_path):
        super().__init__()
        self.model = AutoModel.from_pretrained(
            model_path,
            trust_remote_code=True,
        )

    def forward(
        self,
        input_ids_ref=None,
        input_ids_alt=None,
    ):
        embed_ref = self.model(input_ids=input_ids_ref).last_hidden_state.reshape(len(input_ids_ref), -1)
        embed_alt = self.model(input_ids=input_ids_alt).last_hidden_state.reshape(len(input_ids_ref), -1)
        return F.pairwise_distance(embed_ref, embed_alt)


class ModelVEPAverageRC(torch.nn.Module):
    def __init__(self, model_path):
        super().__init__()
        self.model = AutoModel.from_pretrained(
            model_path,
            trust_remote_code=True,
        )

    def get_scores(self, input_ids_ref, input_ids_alt):
        embed_ref = self.model(input_ids=input_ids_ref).last_hidden_state.reshape(len(input_ids_ref), -1)
        embed_alt = self.model(input_ids=input_ids_alt).last_hidden_state.reshape(len(input_ids_ref), -1)
        return F.pairwise_distance(embed_ref, embed_alt)

    def forward(
        self,
        input_ids_ref_fwd=None,
        input_ids_alt_fwd=None,
        input_ids_ref_rev=None,
        input_ids_alt_rev=None,
    ):
        fwd = self.get_scores(input_ids_ref_fwd, input_ids_alt_fwd)
        rev = self.get_scores(input_ids_ref_rev, input_ids_alt_rev)
        return (fwd + rev) / 2

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
window_size = 512
if average_rc:
    model = ModelVEPAverageRC(model_path)
else:
    model = ModelVEP(model_path)

tokenizer_config.json:   0%|          | 0.00/784 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/419 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/609M [00:00<?, ?B/s]

dilation_schedule=[1, 2, 4, 8, 16, 32, 64, 128, 1, 2, 4, 8, 16, 32, 64, 128, 1, 2, 4, 8, 16, 32, 64, 128, 1, 2, 4, 8, 16, 32, 64, 128, 1, 2, 4, 8, 16, 32, 64, 128, 1, 2, 4, 8, 16, 32, 64, 128, 1, 2, 4, 8, 16, 32, 64, 128, 1, 2, 4, 8, 16, 32, 64, 128]


## Tokenize dataset

In [21]:
class Genome:
    def __init__(self, path):
        self.data = Fasta(fsspec.open(path, anon=True))

    def __call__(self, chrom, start, end, strand="+"):
        res = self.data[chrom][start:end]
        if strand == "-":
            res = res.reverse.complement
        return str(res)

In [25]:
!pip uninstall -y fsspec s3fs
!pip install "fsspec==2024.12.0" "s3fs==2024.12.0"


Found existing installation: fsspec 2024.12.0
Uninstalling fsspec-2024.12.0:
  Successfully uninstalled fsspec-2024.12.0
Found existing installation: s3fs 2025.3.2
Uninstalling s3fs-2025.3.2:
  Successfully uninstalled s3fs-2025.3.2
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.12.0-py3-none-any.whl (183 kB)


In [26]:
genome = Genome("s3://broad-references/hg38/v0/Homo_sapiens_assembly38.fasta")

In [27]:
def tokenize(seqs):
    return tokenizer(
        seqs,
        padding=False,
        truncation=False,
        return_token_type_ids=False,
        return_attention_mask=False,
        return_special_tokens_mask=False,
        return_tensors="pt",
    )["input_ids"]

def get_tokenized_seq(vs):
    # we convert from 1-based coordinate (standard in VCF) to
    # 0-based, to use with Genome
    chrom = np.array(vs["chrom"])
    n = len(chrom)
    pos = np.array(vs["pos"]) - 1
    start = pos - window_size // 2
    end = pos + window_size // 2
    seq_fwd = [genome("chr" + chrom[i], start[i], end[i]) for i in range(n)]
    seq_fwd = np.array([list(seq.upper()) for seq in seq_fwd], dtype="object")
    assert seq_fwd.shape[1] == window_size
    ref_fwd = np.array(vs["ref"])
    alt_fwd = np.array(vs["alt"])
    pos_fwd = window_size // 2

    def prepare_output(seq, pos, ref, alt):
        assert (seq[:, pos] == ref).all(), f"{seq[:, pos]}, {ref}"
        seq_ref = seq
        seq_alt = seq.copy()
        seq_alt[:, pos] = alt
        return (
            tokenize(["".join(x) for x in seq_ref]),
            tokenize(["".join(x) for x in seq_alt]),
        )

    res = {}
    res["input_ids_ref"], res["input_ids_alt"] = prepare_output(seq_fwd, pos_fwd, ref_fwd, alt_fwd)
    return res

def get_tokenized_seq_average_rc(vs):
    # we convert from 1-based coordinate (standard in VCF) to
    # 0-based, to use with Genome
    chrom = np.array(vs["chrom"])
    n = len(chrom)
    pos = np.array(vs["pos"]) - 1
    start = pos - window_size // 2
    end = pos + window_size // 2
    seq_fwd = [genome("chr" + chrom[i], start[i], end[i]) for i in range(n)]
    seq_rev = [str(Seq(x).reverse_complement()) for x in seq_fwd]
    seq_fwd = np.array([list(seq.upper()) for seq in seq_fwd], dtype="object")
    seq_rev = np.array([list(seq.upper()) for seq in seq_rev], dtype="object")
    assert seq_fwd.shape[1] == window_size
    assert seq_rev.shape[1] == window_size
    ref_fwd = np.array(vs["ref"])
    alt_fwd = np.array(vs["alt"])
    ref_rev = np.array([str(Seq(x).reverse_complement()) for x in ref_fwd])
    alt_rev = np.array([str(Seq(x).reverse_complement()) for x in alt_fwd])
    pos_fwd = window_size // 2
    pos_rev = pos_fwd - 1 if window_size % 2 == 0 else pos_fwd

    def prepare_output(seq, pos, ref, alt):
        assert (seq[:, pos] == ref).all(), f"{seq[:, pos]}, {ref}"
        seq_ref = seq
        seq_alt = seq.copy()
        seq_alt[:, pos] = alt
        return (
            tokenize(["".join(x) for x in seq_ref]),
            tokenize(["".join(x) for x in seq_alt]),
        )

    res = {}
    res["input_ids_ref_fwd"], res["input_ids_alt_fwd"] = prepare_output(seq_fwd, pos_fwd, ref_fwd, alt_fwd)
    res["input_ids_ref_rev"], res["input_ids_alt_rev"] = prepare_output(seq_rev, pos_rev, ref_rev, alt_rev)
    return res

In [28]:
if average_rc:
    dataset.set_transform(get_tokenized_seq_average_rc)
else:
    dataset.set_transform(get_tokenized_seq)

In [19]:
dataset[0]

NameError: name 'genome' is not defined

## Run inference

In [16]:
training_args = TrainingArguments(
    output_dir=tempfile.TemporaryDirectory().name,
    per_device_eval_batch_size=128,
    # pyfaidx does not allow multiple workers
    # for longer jobs you can use the gpn.data.Genome class,
    # which loads a local fasta file into memory and can use multiple
    # workers, see e.g.
    # https://github.com/songlab-cal/gpn/blob/main/gpn/ss/run_vep_embed_dist.py
    dataloader_num_workers=0,
    remove_unused_columns=False,
    torch_compile=False,
    fp16=True,
    report_to="none",
)
trainer = Trainer(model=model, args=training_args)
preds = trainer.predict(test_dataset=dataset).predictions
preds.shape

NameError: name 'genome' is not defined

## Compute metrics

In [ ]:
V["score"] = preds

In [ ]:
# let's compare to the precomputed scores
# this is using RC averaging
V["precomputed_score"] = pd.read_parquet("hf://datasets/songlab/TraitGym/mendelian_traits_matched_9/features/GPN_final_EuclideanDistance.parquet").score.values
V[["score", "precomputed_score"]].corr()

In [ ]:
plt.figure(figsize=(2, 2))
sns.histplot(
    data=V, x="score", bins=30, hue="label", stat="density",
    common_norm=False, common_bins=True,
)
sns.despine();

In [ ]:
# global AUPRC
average_precision_score(V.label, V.score)

In [ ]:
# AUPRC by chrom
res_by_chrom = []
for chrom in V.chrom.unique():
    V_chrom = V[V.chrom == chrom]
    res_by_chrom.append([chrom, len(V_chrom), average_precision_score(V_chrom.label, V_chrom.score)])
res_by_chrom = pd.DataFrame(res_by_chrom, columns=["chrom", "n", "AUPRC"])
res_by_chrom

In [ ]:
# Weighted average

def stat(df):
    weight = df["n"] / df["n"].sum()
    return (df["AUPRC"] * weight).sum()

stat(res_by_chrom)

In [ ]:
def bootstrap_se(df, stat, n_bootstraps=1000):
    df = pl.DataFrame(df)
    return (
        pl.Series([
            stat(df.sample(len(df), with_replacement=True, seed=i))
            for i in range(n_bootstraps)]
        )
        .std()
    )

bootstrap_se(res_by_chrom, stat)

In [ ]:
# compare with official results (which use RC averaging)
pd.read_csv("hf://datasets/songlab/TraitGym/mendelian_traits_matched_9/AUPRC_by_chrom_weighted_average/all/GPN_final_EuclideanDistance.plus.score.csv")